In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
path = '../Data/Preprocessed/topicResult_10_summary_tags.csv'
df_origin = pd.read_csv(path, encoding = 'cp949')
df_origin['date'] = pd.to_datetime(df_origin['date'])
df_origin['news_id'] = df_origin['news_id'].astype(str)
df_origin['propTopic'] = df_origin['propTopic'].astype(str)

innerOuter_path = '../Data/Preprocessed/innerOutTaged.csv'
innerOuter_origin = pd.read_csv(innerOuter_path)
innerOuter_origin['Timestamp'] = pd.to_datetime(innerOuter_origin['Timestamp'])
innerOuter_origin['news_id'] = innerOuter_origin['URL'].str.extract(r'news_id=(\d*)')
innerOuter_origin = innerOuter_origin.merge(df_origin[['news_id', 'propTopic']], on = ['news_id'], how = 'left')

C:\Users\WESLEYQUEST\AppData\Local\Temp\ipykernel_836\1367436339.py:8: DtypeWarning: Columns (3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  innerOuter_origin = pd.read_csv(innerOuter_path)


In [3]:
df_origin.head(1)

,title,sub-title,publisher,author,date,text,keywords,href,numKeywords,news_id,...,day,summary,summary_tags,title_tags,subtitle_tags,keywords_tags,numTopic,propTopic,propTopicRatio,topicsRatio
0,2022년 올해의 정원식물은?,‘2022 대한민국 정원식물 전시·품평회’ 성료,라펜트,전지은,2022-11-07,‘2022 대한민국 정원식물 전시·품평회’를? 9월 30일부터 10월 30일까지 개...,"['한국수목원정원관리원', '대한민국 정원식물 전시·품평회', '정원식물']",https://www.lafent.com/inews/news_view.html?ne...,3,131470,...,7.0,한국수목원정원관리원은 가을 정취를 느낄 수 있는 FW 시즌 정원식물 788품종을 한...,"['한국', '수목원', '정원', '관리원', '가을', '정취', '느끼다', ...","['올해', '정원', '식물']","['대한민국', '정원', '식물', '전시', '품평회', '성']","['한국', '수목원', '정원', '관리원', '대한민국', '정원', '식물',...",10,8,0.701,"[(0, 0.12626038), (4, 0.07635856), (5, 0.08301..."


In [5]:
innerOuter_origin.head(1)

,Timestamp,IP,URL,URL Search Word,Referer,Referer Search Word,Referer Source,Referer Category,New IP,User Type,month,day,hour,bigPath,middlePath,smallPath,tmp,news_id,propTopic
0,2022-08-01 14:30:55+09:00,1.220.249.83,/jobse/job_view.html?mcd=c01&idx=24596,NaN,NaN,NaN,NaN,etc,10,내부,8,1,14,채용/인재,시공/감리/생산,메인,jobse,NaN,NaN


In [77]:
# 토픽별 게시글 수
df_origin.groupby(['propTopic'])['href'].nunique().reset_index()

,propTopic,href
0,0,242
1,1,1824
2,2,255
3,3,204
4,4,291
5,5,220
6,6,1217
7,7,749
8,8,1053
9,9,1001


In [78]:
# 토픽별 조회수
inner_df = innerOuter_origin.copy()
tmp01 = inner_df.loc[inner_df['news_id'].notnull()].groupby(['propTopic', 'User Type'])['URL'].count().reset_index().pivot_table(index = ['propTopic'], columns = ['User Type'], values = ['URL']).fillna(0)
tmp01['total'] = tmp01.sum(axis = 1)
tmp01

URL        total
User Type     내부    외부       
propTopic                    
0           7201  3187  10388
1          34022  7778  41800
2           5187   716   5903
3           4827   658   5485
4           6621  2234   8855
5           5868   938   6806
6          34184  3169  37353
7          23179  3690  26869
8          26747  4827  31574
9          20073  2811  22884

In [79]:
inner_df = innerOuter_origin.copy()
tmp02 = inner_df.loc[inner_df['news_id'].notnull()].groupby(['propTopic', 'User Type'])['New IP'].nunique().reset_index().pivot_table(index = ['propTopic'], columns = ['User Type'], values = ['New IP']).fillna(0)
tmp02['total'] = tmp02.sum(axis = 1)
tmp02

New IP        total
User Type     내부    외부       
propTopic                    
0           2532  2857   5389
1           7913  6085  13998
2           2047   523   2570
3           1658   491   2149
4           2412  1868   4280
5           1950   690   2640
6           7082  1792   8874
7           5315  2612   7927
8           6556  3392   9948
9           5227  2093   7320

In [81]:
tmp01.columns = ['내부', '외부', '전체']
tmp02.columns = ['내부', '외부', '전체']
tmp01

,내부,외부,전체
propTopic,,,
0,7201,3187,10388
1,34022,7778,41800
2,5187,716,5903
3,4827,658,5485
4,6621,2234,8855
5,5868,938,6806
6,34184,3169,37353
7,23179,3690,26869
8,26747,4827,31574


In [99]:
tmp = round(tmp01 / tmp02, 2).T
tmp.loc['비교'] = round(tmp.loc['외부'] / tmp.loc['내부'] * 100 , 2)
tmp

propTopic,0,1,2,3,4,5,6,7,8,9
내부,2.84,4.30,2.53,2.91,2.75,3.01,4.83,4.36,4.08,3.84
외부,1.12,1.28,1.37,1.34,1.20,1.36,1.77,1.41,1.42,1.34
전체,1.93,2.99,2.30,2.55,2.07,2.58,4.21,3.39,3.17,3.13
비교,39.44,29.77,54.15,46.05,43.64,45.18,36.65,32.34,34.80,34.90


: 

In [4]:
# 토픽별 전체 게시글 비율
tmp = df_origin.groupby(['propTopic'])['href'].count()
tmp = round(tmp/tmp.sum() * 100, 2).reset_index()
tmp

,propTopic,href
0,0,3.43
1,1,25.85
2,2,3.61
3,3,2.89
4,4,4.12
5,5,3.12
6,6,17.25
7,7,10.62
8,8,14.92
9,9,14.19


In [15]:
innerOuter_origin.head(2)

,Timestamp,IP,URL,URL Search Word,Referer,Referer Search Word,Referer Source,Referer Category,New IP,User Type,month,day,hour,bigPath,middlePath,smallPath,tmp,news_id,propTopic
0,2022-08-01 14:30:55+09:00,1.220.249.83,/jobse/job_view.html?mcd=c01&idx=24596,NaN,NaN,NaN,NaN,etc,10,내부,8,1,14,채용/인재,시공/감리/생산,메인,jobse,NaN,NaN
1,2022-08-01 14:32:14+09:00,1.220.249.83,/jobse/job_view.html?mcd=c01&idx=23934,NaN,NaN,NaN,NaN,etc,10,내부,8,1,14,채용/인재,시공/감리/생산,메인,jobse,NaN,NaN


In [29]:
df_origin.columns

Index(['title', 'sub-title', 'publisher', 'author', 'date', 'text', 'keywords',
       'href', 'numKeywords', 'news_id', 'year', 'month', 'day', 'summary',
       'summary_tags', 'title_tags', 'subtitle_tags', 'keywords_tags',
       'numTopic', 'propTopic', 'propTopicRatio', 'topicsRatio'],
      dtype='object')

In [34]:
# 뉴스기사 토픽 및 월별 최다 조회수 (해당 월별 집계)
tmp = innerOuter_origin.loc[innerOuter_origin['propTopic'].notnull()]
tmp = tmp.groupby(['propTopic', 'month', 'news_id']).agg({'URL' : 'count',
                                                          'New IP' : 'nunique'}).reset_index()
tmp = tmp.groupby(['propTopic', 'month']).apply(lambda x : x.sort_values(by = ['URL', 'New IP'], ascending = False).iloc[:5]).reset_index(drop = True)
tmp = tmp.merge(df_origin.loc[:, ['news_id', 'title', 'date']], on = 'news_id', how = 'left')
tmp = tmp.drop(columns = ['news_id'])
tmp = tmp[['propTopic', 'month', 'title', 'date', 'URL', 'New IP']]
tmp.to_csv('../Data/Result/뉴스기사 토픽 및 월별 최다 조회수.csv', index = False, encoding = 'utf-8-sig', escapechar = '\\')

In [35]:
tmp

,propTopic,month,title,date,URL,New IP
0,0,1,라펜트가 선정한 2022년 조경계 10대뉴스 - 해외편,2022-12-30,175,39
1,0,1,"순천만국제정원박람회 그 후 10년, 변화상에 귀추 ‘주목’",2023-01-08,91,35
2,0,1,"청와대 권역 담장, 경복궁 후원 담장 위치와 일치",2023-01-04,75,21
3,0,1,공원만족도의 결정적 요인은? ‘보행’과 ‘수목’,2022-07-11,45,25
4,0,1,"용인시, 심곡서원 역사공원 조성한다",2022-12-01,36,18
...,...,...,...,...,...,...
595,9,12,尹정부 첫 도시재생사업지 26곳 선정···’27년까지 1.5조원 투자,2022-12-16,196,61
596,9,12,"인사처, 국가공무원 조경직 7명 선발···예년대비 절반가량 축소",2022-12-21,143,31
597,9,12,"1천가구 도시재생 사업, 공원·녹지 의무확보 없어져",2022-12-01,114,64
598,9,12,‘지각 예산’ 국토부?환경부 지출 규모 확정…국민 안전 등 투자 확대,2022-12-27,73,16
